# Building Web Applications with Python Shiny

This tutorial will guide you through using Python Shiny to build interactive web applications and dashboards. We'll explore Shiny's design philosophy, reactive programming model, main components, and best practices for building professional dashboards.

## What is Python Shiny?

Python Shiny is a web framework that makes it easy to build interactive web applications straight from Python. It's inspired by R Shiny but designed specifically for Python developers, bringing reactive programming and modern web development patterns to the Python ecosystem.

### Design Philosophy

Python Shiny is built around several core principles:

1. **Reactive Programming**: Automatic updates when inputs change - no manual event handling
2. **Python-First**: Leverage the entire Python ecosystem (pandas, matplotlib, plotly, etc.)
3. **Separation of Concerns**: Clear distinction between UI definition and server logic
4. **Modern Web Standards**: Built on modern async Python and web technologies
5. **Flexibility**: From simple apps to complex dashboards with custom layouts
6. **Performance**: Efficient reactive updates and async support

### When to Use Python Shiny

Python Shiny excels for:
- Interactive data dashboards
- Business intelligence applications
- Scientific computing interfaces
- Real-time monitoring systems
- Data exploration tools
- Machine learning model interfaces
- Complex multi-page applications

## Installation and Setup

First, let's install Python Shiny and import the necessary libraries:

In [2]:
# Install Python Shiny (run this in terminal if not already installed)
# pip install shiny

from shiny import App, Inputs, Outputs, Session, reactive, render, ui
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

## Core Concepts

### 1. The Shiny App Structure

Every Shiny app has two main components:
- **UI (User Interface)**: Defines the layout and inputs/outputs
- **Server**: Contains the reactive logic and computations

Let's start with a simple "Hello World" example:

In [3]:
# Simple Hello World App
app_ui = ui.page_fluid(
    ui.h2("Hello Shiny!"),
    ui.input_text("name", "Enter your name:", value="World"),
    ui.output_text_verbatim("greeting")
)

def server(input: Inputs, output: Outputs, session: Session):
    @output
    @render.text
    def greeting():
        return f"Hello, {input.name()}!"

# Create the app
hello_app = App(app_ui, server)

# To run the app, use: hello_app.run()
print("Hello World app created. Run with: hello_app.run()")

Hello World app created. Run with: hello_app.run()


### 2. Reactive Programming Model

The key to Shiny's power is its reactive programming model:

- **Reactive Sources**: Inputs that can change (user inputs, file uploads, etc.)
- **Reactive Conductors**: Intermediate calculations that depend on sources
- **Reactive Endpoints**: Outputs that display results (plots, tables, text)

When a reactive source changes, all dependent conductors and endpoints automatically update.

## Main Components

### UI Components

#### Layout Functions
- `ui.page_fluid()`: Responsive fluid layout
- `ui.page_fixed()`: Fixed-width layout
- `ui.row()`: Horizontal row
- `ui.column()`: Column within a row
- `ui.sidebar()`: Sidebar layout

#### Input Components
- `ui.input_text()`: Text input
- `ui.input_numeric()`: Numeric input
- `ui.input_slider()`: Slider input
- `ui.input_select()`: Dropdown selection
- `ui.input_checkbox()`: Checkbox
- `ui.input_radio_buttons()`: Radio buttons
- `ui.input_file()`: File upload
- `ui.input_date()`: Date picker

#### Output Components
- `ui.output_text()`: Text output
- `ui.output_plot()`: Plot output
- `ui.output_table()`: Table output
- `ui.output_data_frame()`: Interactive data frame
- `ui.output_image()`: Image output
- `ui.output_ui()`: Dynamic UI output

## Practical Examples

### Example 1: Interactive Data Explorer

Let's create a data exploration app using the penguins dataset:

In [ ]:
# Load the penguins dataset
penguins = pd.read_csv('data/penguins.csv')

# Data Explorer App
explorer_ui = ui.page_fluid(
    ui.h1("🐧 Penguin Data Explorer"),
    ui.p("Explore the Palmer Penguins dataset with interactive visualizations"),
    
    ui.layout_sidebar(
        ui.panel_sidebar(
            ui.h3("Controls"),
            ui.input_select(
                "species",
                "Select Species:",
                choices=["All"] + list(penguins['species'].unique()),
                selected="All"
            ),
            ui.input_select(
                "x_var",
                "X-axis Variable:",
                choices=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'],
                selected='bill_length_mm'
            ),
            ui.input_select(
                "y_var",
                "Y-axis Variable:",
                choices=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'],
                selected='bill_depth_mm'
            ),
            ui.input_checkbox("show_trend", "Show Trend Line", value=False),
            ui.br(),
            ui.input_action_button("reset", "Reset Filters", class_="btn-secondary")
        ),
        
        ui.panel_main(
            ui.navset_tab(
                ui.nav(
                    "Scatter Plot",
                    ui.output_plot("scatter_plot", height="500px")
                ),
                ui.nav(
                    "Summary Statistics",
                    ui.output_data_frame("summary_stats")
                ),
                ui.nav(
                    "Data Table",
                    ui.output_data_frame("data_table")
                )
            )
        )
    )
)

def explorer_server(input: Inputs, output: Outputs, session: Session):
    
    @reactive.Calc
    def filtered_data():
        data = penguins.copy()
        if input.species() != "All":
            data = data[data['species'] == input.species()]
        return data
    
    @output
    @render.plot
    def scatter_plot():
        data = filtered_data()
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        if input.species() == "All":
            sns.scatterplot(
                data=data, 
                x=input.x_var(), 
                y=input.y_var(), 
                hue='species',
                ax=ax
            )
        else:
            sns.scatterplot(
                data=data, 
                x=input.x_var(), 
                y=input.y_var(),
                ax=ax
            )
        
        if input.show_trend():
            sns.regplot(
                data=data, 
                x=input.x_var(), 
                y=input.y_var(), 
                scatter=False, 
                ax=ax,
                color='red'
            )
        
        ax.set_title(f'{input.y_var()} vs {input.x_var()}')
        plt.tight_layout()
        return fig
    
    @output
    @render.data_frame
    def summary_stats():
        data = filtered_data()
        numeric_cols = data.select_dtypes(include=[np.number]).columns
        return data[numeric_cols].describe()
    
    @output
    @render.data_frame
    def data_table():
        return filtered_data()
    
    @reactive.Effect
    @reactive.event(input.reset)
    def reset_filters():
        ui.update_select("species", selected="All")
        ui.update_select("x_var", selected="bill_length_mm")
        ui.update_select("y_var", selected="bill_depth_mm")
        ui.update_checkbox("show_trend", value=False)

penguin_explorer = App(explorer_ui, explorer_server)
print("Penguin Explorer app created. Run with: penguin_explorer.run()")

### Example 2: Simple Calculator Dashboard

Let's create a calculator app to demonstrate reactive programming:

In [ ]:
# Calculator Dashboard
calc_ui = ui.page_fluid(
    ui.h1("🧮 Interactive Calculator"),
    ui.p("Perform calculations with real-time updates"),
    
    ui.row(
        ui.column(
            4,
            ui.card(
                ui.card_header("Input Values"),
                ui.card_body(
                    ui.input_numeric("num1", "First Number:", value=10),
                    ui.input_select(
                        "operation",
                        "Operation:",
                        choices=["+", "-", "*", "/", "**"],
                        selected="+"
                    ),
                    ui.input_numeric("num2", "Second Number:", value=5),
                )
            )
        ),
        
        ui.column(
            8,
            ui.card(
                ui.card_header("Results"),
                ui.card_body(
                    ui.div(
                        ui.h2(ui.output_text("calculation")),
                        style="text-align: center; padding: 2rem; background: #f8f9fa; border-radius: 10px;"
                    ),
                    ui.br(),
                    ui.output_plot("history_plot")
                )
            )
        )
    )
)

def calc_server(input: Inputs, output: Outputs, session: Session):
    # Store calculation history
    history = reactive.Value([])
    
    @reactive.Calc
    def current_result():
        try:
            num1 = input.num1() or 0
            num2 = input.num2() or 0
            op = input.operation()
            
            if op == "+":
                result = num1 + num2
            elif op == "-":
                result = num1 - num2
            elif op == "*":
                result = num1 * num2
            elif op == "/":
                result = num1 / num2 if num2 != 0 else float('inf')
            elif op == "**":
                result = num1 ** num2
            
            # Update history
            current_history = history.get()
            current_history.append(result)
            if len(current_history) > 20:  # Keep last 20 results
                current_history = current_history[-20:]
            history.set(current_history)
            
            return result
        except:
            return "Error"
    
    @output
    @render.text
    def calculation():
        result = current_result()
        num1 = input.num1() or 0
        num2 = input.num2() or 0
        op = input.operation()
        
        if result == "Error":
            return "Error in calculation"
        
        return f"{num1} {op} {num2} = {result}"
    
    @output
    @render.plot
    def history_plot():
        hist = history.get()
        
        if len(hist) < 2:
            fig, ax = plt.subplots(figsize=(8, 4))
            ax.text(0.5, 0.5, 'No calculation history yet', 
                   horizontalalignment='center', verticalalignment='center',
                   transform=ax.transAxes, fontsize=14)
            ax.set_xlim(0, 1)
            ax.set_ylim(0, 1)
            ax.axis('off')
            return fig
        
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.plot(range(len(hist)), hist, marker='o', linewidth=2, markersize=6)
        ax.set_title('Calculation History')
        ax.set_xlabel('Calculation Number')
        ax.set_ylabel('Result')
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        return fig

calculator_app = App(calc_ui, calc_server)
print("Calculator app created. Run with: calculator_app.run()")

## Advanced Features

### 1. Reactive Programming Patterns

#### Reactive Calculations
Use `@reactive.Calc` for expensive computations that should be cached:

```python
@reactive.Calc
def expensive_computation():
    # This will only run when dependencies change
    return heavy_processing(input.data())
```

#### Reactive Effects
Use `@reactive.Effect` for side effects (logging, saving files, etc.):

```python
@reactive.Effect
def log_user_action():
    print(f"User selected: {input.choice()}")
```

#### Event Handling
Use `@reactive.event` to respond to specific events:

```python
@reactive.Effect
@reactive.event(input.button_click)
def handle_button_click():
    # Only runs when button is clicked
    process_data()
```

## Building Production Dashboards

### 1. Professional Dashboard Example

Let's create a comprehensive business dashboard:

In [ ]:
# Professional Sales Dashboard
dashboard_ui = ui.page_fluid(
    ui.tags.head(
        ui.tags.title("Sales Analytics Dashboard"),
        ui.tags.style("""
        .dashboard-header {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 2rem;
            margin-bottom: 2rem;
            border-radius: 10px;
        }
        .metric-card {
            background: white;
            border: 1px solid #e9ecef;
            border-radius: 10px;
            padding: 1.5rem;
            margin-bottom: 1rem;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            transition: transform 0.2s;
        }
        .metric-card:hover {
            transform: translateY(-2px);
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }
        .metric-value {
            font-size: 2.5rem;
            font-weight: bold;
            color: #495057;
            margin: 0;
        }
        """)
    ),
    
    ui.div(
        ui.h1("📊 Sales Analytics Dashboard", class_="mb-0"),
        ui.p("Real-time business intelligence and performance metrics", class_="mb-0"),
        class_="dashboard-header text-center"
    ),
    
    ui.row(
        ui.column(
            3,
            ui.div(
                ui.h4("🎛️ Controls"),
                ui.input_date_range(
                    "date_range",
                    "Date Range:",
                    start="2023-01-01",
                    end="2023-12-31"
                ),
                ui.input_selectize(
                    "regions",
                    "Regions:",
                    choices=["North", "South", "East", "West", "Central"],
                    selected=["North", "South", "East", "West", "Central"],
                    multiple=True
                ),
                ui.input_action_button(
                    "refresh_data",
                    "🔄 Refresh Data",
                    class_="btn-primary w-100"
                ),
                class_="metric-card"
            )
        ),
        
        ui.column(
            9,
            ui.row(
                ui.column(3, ui.output_ui("total_revenue_card")),
                ui.column(3, ui.output_ui("total_orders_card")),
                ui.column(3, ui.output_ui("avg_order_value_card")),
                ui.column(3, ui.output_ui("customer_count_card"))
            ),
            
            ui.row(
                ui.column(
                    8,
                    ui.div(
                        ui.h4("📈 Revenue Trend"),
                        ui.output_ui("revenue_chart"),
                        class_="metric-card"
                    )
                ),
                ui.column(
                    4,
                    ui.div(
                        ui.h4("🥧 Sales by Region"),
                        ui.output_ui("region_chart"),
                        class_="metric-card"
                    )
                )
            )
        )
    )
)

def dashboard_server(input: Inputs, output: Outputs, session: Session):
    
    @reactive.Calc
    def sample_sales_data():
        # Generate sample sales data
        np.random.seed(42)
        n_records = 5000
        
        dates = pd.date_range('2023-01-01', '2023-12-31', freq='D')
        
        data = []
        for _ in range(n_records):
            date = np.random.choice(dates)
            region = np.random.choice(['North', 'South', 'East', 'West', 'Central'])
            customer_id = f"CUST_{np.random.randint(1000, 9999)}"
            order_value = np.random.lognormal(5, 1)
            
            data.append({
                'date': date,
                'region': region,
                'customer_id': customer_id,
                'order_value': order_value
            })
        
        return pd.DataFrame(data)
    
    @reactive.Calc
    def filtered_data():
        data = sample_sales_data()
        data = data[data['region'].isin(input.regions())]
        
        start_date = pd.to_datetime(input.date_range()[0])
        end_date = pd.to_datetime(input.date_range()[1])
        data = data[(data['date'] >= start_date) & (data['date'] <= end_date)]
        
        return data
    
    @output
    @render.ui
    def total_revenue_card():
        data = filtered_data()
        total_revenue = data['order_value'].sum()
        
        return ui.div(
            ui.div(f"${total_revenue:,.0f}", class_="metric-value"),
            ui.div("💰 Total Revenue"),
            class_="metric-card text-center"
        )
    
    @output
    @render.ui
    def total_orders_card():
        data = filtered_data()
        total_orders = len(data)
        
        return ui.div(
            ui.div(f"{total_orders:,}", class_="metric-value"),
            ui.div("📦 Total Orders"),
            class_="metric-card text-center"
        )
    
    @output
    @render.ui
    def avg_order_value_card():
        data = filtered_data()
        avg_order = data['order_value'].mean() if len(data) > 0 else 0
        
        return ui.div(
            ui.div(f"${avg_order:.0f}", class_="metric-value"),
            ui.div("💵 Avg Order Value"),
            class_="metric-card text-center"
        )
    
    @output
    @render.ui
    def customer_count_card():
        data = filtered_data()
        unique_customers = data['customer_id'].nunique()
        
        return ui.div(
            ui.div(f"{unique_customers:,}", class_="metric-value"),
            ui.div("👥 Customers"),
            class_="metric-card text-center"
        )
    
    @output
    @render.ui
    def revenue_chart():
        data = filtered_data()
        
        if len(data) == 0:
            return ui.p("No data available")
        
        daily_revenue = data.groupby('date')['order_value'].sum().reset_index()
        
        fig = px.line(
            daily_revenue,
            x='date',
            y='order_value',
            title="Daily Revenue Trend",
            height=300
        )
        fig.update_layout(showlegend=False)
        
        return ui.HTML(fig.to_html(full_html=False, include_plotlyjs="cdn"))
    
    @output
    @render.ui
    def region_chart():
        data = filtered_data()
        
        if len(data) == 0:
            return ui.p("No data available")
        
        region_sales = data.groupby('region')['order_value'].sum().reset_index()
        
        fig = px.pie(
            region_sales,
            values='order_value',
            names='region',
            title="Sales by Region",
            height=300
        )
        
        return ui.HTML(fig.to_html(full_html=False, include_plotlyjs="cdn"))

sales_dashboard = App(dashboard_ui, dashboard_server)
print("Sales Dashboard created. Run with: sales_dashboard.run()")

## Deployment and Production

### 1. Local Development

Run your Shiny app locally:

```python
# Basic launch
app.run()

# With custom settings
app.run(
    host="0.0.0.0",  # Allow external connections
    port=8000,       # Custom port
    reload=True,     # Auto-reload on code changes
    launch_browser=True  # Open browser automatically
)
```

### 2. Cloud Deployment Options

#### Shinyapps.io
The easiest deployment option for Shiny apps:

```bash
# Install rsconnect-python
pip install rsconnect-python

# Deploy to shinyapps.io
rsconnect deploy shiny /path/to/your/app --name your-app-name
```

#### Docker Deployment
Create a Dockerfile for containerized deployment:

```dockerfile
FROM python:3.11

WORKDIR /app

COPY requirements.txt .
RUN pip install -r requirements.txt

COPY . .

EXPOSE 8000

CMD ["python", "-m", "shiny", "run", "app.py", "--host", "0.0.0.0", "--port", "8000"]
```

### 3. Best Practices

#### Code Organization
```
my_shiny_app/
├── app.py              # Main app file
├── ui/
│   ├── __init__.py
│   └── components.py   # UI components
├── server/
│   ├── __init__.py
│   └── logic.py       # Server logic
├── utils/
│   └── helpers.py     # Helper functions
└── requirements.txt
```

#### Performance Optimization
- Use `@reactive.Calc` for expensive computations
- Implement caching for data that doesn't change often
- Use async operations for I/O bound tasks
- Minimize reactive dependencies

## Running the Apps

To run any of the apps we've created, use:

```python
# Choose which app to run:
# hello_app.run()
# penguin_explorer.run()
# calculator_app.run()
sales_dashboard.run(host="0.0.0.0", port=8000, reload=True)
```

The app will be available at `http://localhost:8000`.

## Conclusion

Python Shiny provides a powerful framework for building interactive web applications with Python. Key takeaways:

### ✅ Advantages of Python Shiny:
- **Reactive Programming**: Automatic updates with minimal boilerplate
- **Python Ecosystem**: Full access to pandas, matplotlib, plotly, etc.
- **Modern Web Standards**: Built on async Python and modern web technologies
- **Flexible Layouts**: From simple apps to complex multi-page dashboards
- **Production Ready**: Robust deployment options and performance optimization
- **Type Safety**: Built-in type hints and IDE support

### 🎯 Best Use Cases:
- Business intelligence dashboards
- Data exploration and analysis tools
- Scientific computing interfaces
- Real-time monitoring systems
- Machine learning model interfaces
- Complex multi-page applications

### 🚀 Next Steps:
1. Practice with the examples in this notebook
2. Build your own dashboard using your data
3. Explore advanced features like custom CSS and JavaScript integration
4. Deploy your applications to production
5. Contribute to the Python Shiny community

### 📚 Additional Resources:
- [Python Shiny Documentation](https://shiny.posit.co/py/)
- [Python Shiny GitHub Repository](https://github.com/posit-dev/py-shiny)
- [Shiny for Python Gallery](https://shiny.posit.co/py/gallery/)
- [Posit Community](https://community.rstudio.com/c/shiny/)

Python Shiny brings the power of reactive programming to Python, making it easier than ever to build sophisticated, interactive web applications for data science and beyond. Happy building! 🎉